<a href="https://colab.research.google.com/github/RitaRez/POC/blob/main/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas Utilizadas e Mount do Drive

In [2]:
!pip install num2words
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=af80602944c61b0cc4c94cb66e12b8e6a3faaadcb816a6eb334ae9a41a3bce27
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import json, re, nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = "/content/drive/My Drive/ToT/"
!ls "/content/drive/My Drive/ToT"

Index  Movies


# Leitura de Corpus e Consultas

In [5]:
with open(path + 'Movies/queries.json', 'rt') as file:
  
  queries = []
  for l in file:
      queries.append(json.loads(l))

queries[0]

{'id': 'cggmzb',
 'title': '[TOMT][ANIMATION] Little girl turns out to be a robot. 2000s?',
 'description': 'Hi,\n\n&#x200B;\n\nAs a kid I saw probably a middle scene in a animated film in which a girl turns out to be a robot, feels betrayed by humans and joins other machines to fight humanity. I only saw a few scenes. Looked a lot like a Ghibli film.',
 'url': 'https://www.reddit.com/r/tipofmytongue/comments/cggmzb/tomtanimation_little_girl_turns_out_to_be_a_robot/'}

In [6]:
with open(path + 'Movies/documents.json', 'rt') as file:
  
  docs = []
  for l in file:
      docs.append(json.loads(l))

docs[0]

{'id': 'tt0478303',
 'text': 'Seven strangers visiting Hollywood movie studios are locked inside an ill-famed House of Horror.\nTo leave the trap alive, they have to tell their most terrifying stories.',
 'title': 'Trapped Ashes',
 'meta': {'wikidata_id': 'Q3538057'}}

# Qual a proporção de consultas que apresentam referencias temporais?

É importante lembrar que esses números podem se referir a época onde se passa o filme ou até mesmo números que não representam datas.

In [7]:
numeric_time_references = r"\b(20)?(\d{2})[']?s?\b|\b(20)?(\d{2})\b"

print(queries[4]['title'], " Match: ", re.search(numeric_time_references, queries[4]['title']))
print(queries[5]['title'], " Match: ", re.search(numeric_time_references, queries[5]['title']))

[TOMT] [MOVIE] Horror film based on true story which I'm almost certain starred Richard Thomas from The Waltons  Match:  None
[TOMT][MOVIE][Early 2000s]Movie in which two perfect strangers have casual sex but everything ends when one of the two tries to know more about the other  Match:  <re.Match object; span=(20, 25), match='2000s'>


In [23]:
count = 0
for query in queries:
  if re.search(numeric_time_references, query['title']):
    count += 1

print(f"{100*count/len(queries)}% of queries have some form of time frame in the title")

count = 0
for query in queries:
  if  re.search(numeric_time_references, query['description']):
    count += 1

print(f"{100*count/len(queries)}% of queries have some form of time frame in the description")

count = 0
for query in queries:
  if re.search(numeric_time_references, query['description']) or re.search(numeric_time_references, querie['title']):
    count += 1

print(f"{100*count/len(queries)}% of queries have some form of time frame in the title or the description")

41.79669395952418% of queries have some form of time frame in the title
41.75807199134868% of queries have some form of time frame in the description
100.0% of queries have some form of time frame in the title or the description


# Qual a proporção de consultas que apresentam referências a nomes próprios?

Podem ser referencias a atores, diretores, personagens...

In [22]:
nltk_results = ne_chunk(pos_tag(word_tokenize(queries[1]['description'])))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        if nltk_result.label() == 'PERSON':
          print ('Type: ', nltk_result.label(), 'Name: ', name)


Type:  PERSON Name:  Born 
Type:  PERSON Name:  Matilda 
Type:  PERSON Name:  Matilda 


In [26]:
count = 0
for query in queries:
  nltk_results = ne_chunk(pos_tag(word_tokenize(query['title'] + query['description'])))
  for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
      name = ''
      for nltk_result_leaf in nltk_result.leaves():
        name += nltk_result_leaf[0] + ' '
      if nltk_result.label() == 'PERSON':
        count += 1

print(f"{100*count/len(queries)}% of queries have some form of person name in the title or the description")

67.48030279623049% of queries have some form of person name in the title or the description
